### <img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393/578 Máquinas de Aprendizaje - 2019-1 </h1>

<H3 align='center'> Tarea 3  </H3>
<hr style="height:2px;border:none"/>

**Temas**  
* Manipulaciones en numpy
* Arboles de clasificación y regresión
* Algunas Redes neuronales básicas
* Ensamblados, Adaboost y Random Forest
* Nociones de clases desbalanceadas


**Formalidades**  
* Equipos de trabajo de 2 personas (*Ambos estudiantes deben estar preparados para presentar la tarea el día de la entrega*)
* El entregable debe ser un _Jupyter Notebook_ incluyendo los códigos utilizados, los resultados, los gráficos realizados y comentarios. Debe seguir una estructura similar a un informe (se debe introducir los problemas a trabajar, presentar los resultados y discutirlos). Si lo prefiere puede entregar un _Jupyter Notebook_ por pregunta o uno por toda la tarea, con tal de que todos los entregables esten bien identificados y se encuentren en el mismo repositorio de _Github_.
* Se debe preparar una presentación del trabajo realizado y sus hallazgos. El presentador será elegido aleatoriamente y deberá apoyarse en el _Jupyter Notebook_ que entregarán. 
* Formato de entrega: envı́o de link del repositorio *privado* en _Github_, al correo electrónico del ayudante (*<alvaro.valderrama.13@sansano.usm.cl>*), en copia al profesor (*<cvalle@inf.utfsm.cl>*). Especificar el siguiente asunto: [INF393/578-2019 Tarea 3]. Invitar como colaborador al usuario de github "avalderr" para poder acceder al repositorio.
* Fecha de entrega y presentaciones: 13 de Septiembre. Hora límite de entrega: 23:50. Cualquier _commit_ luego de la hora límite no será evaluado. Se realizará descuento por atrasos en envío del mail. 

<hr style="height:2px;border:none"/>

La tarea se divide en tres partes:

[1.](#primero) Regresión para ubicación espacial  
[2.](#segundo) Clasificación con clases desbalanceadas    
[3.](#tercero) Clasificación en texto   

La tarea tiene ejemplos de códigos con los cuales pueden guiarse en gran parte, sin embargo solo son guias y pueden ser creativos al momento de resolver la tarea. Soluciones creativas o elegantes serán valoradas. También en algunas ocaciones se hacen elecciones arbitrarias, ustedes pueden realizar otras elecciones con tal de que haya una pequeña justificación de por qué su elección es mejor o equivalente.
Recuerden intercalar su código con comentarios y con celdas _Markdown_ con los comentarios de la pregunta y con cualquier analisis, fórmula (en $ \LaTeX $) o explicación que les parezca relevante para justificar sus procedimientos. 
Noten que en general cuando se les pide elegir algo o proponer algo no se evaluará mucho la elección en si, en cambio la argumentación detrás de la elección será lo más ponderado.
Si algun modelo se demora demasiado en correr en su maquina, no olvide que puede correr _Jupyter Notebooks_ en _Collab_ de Google, esto puede ser relevante para las maquinas más lentas al momento de realizar exploraciones con _K-folds_ por ejemplo.

<a id="segundo"></a>
## 2. Clasificación con clases desbalanceadas

En las tareas de clasificación supervisada, buscamos que mediante los ejemplos etiquetados la maquina pueda aprender los atributos inerentes a las distintas clases. Sin embargo, en muchos casos reales de clasificación, la cantidad de ejemplos de cada clase son muy dispares, en cuyo caso hablamos de clases desbalanceadas. Si uno no aplica estrategias para compensar este problema, la maquina aprenderá muy bien las caracteristicas de la clase más representada pero no logrará extraer información generalizable de la clase menos representada. Además, debemos tener cuidado al momento de interpretar las distintas métricas a las cuales tenemos acceso, pues debemos tomar en cuenta el balance de las clases y la naturaleza del problema para evaluar realmente que tan bueno es el desempeño. 



Para esta parte de la tarea utilizaremos el siguiente _dataset_ publicado en Kaggle: https://www.kaggle.com/mlg-ulb/creditcardfraud. Este conjunto de datos contiene unos 285000 ejemplos de transacciones con tarjetas de crédito reales, realizadas en 2013 por clientes europeos. Los datos son totalmente anónimos y son el resultado de un PCA a partir de los datos originales excepto por las columnas `Time` con el tiempo en segundos desde la primera transacción, `Amount` con el monto de la transacción y `Class` que indica si la transacción es fraudulenta o no. La tarea en cuestión consiste en lograr predecir cuando una transacción es fraudulenta o no automaticamente, para así poder detenerla antes de que se termine. Sin embargo, por la naturaleza de las transacciones bancarias, este problema es desbalanceado, de hecho de los 285000 transacciones registradas, solo 492 son fraudulentas! A lo largo de esta pregunta pondremos en evidencia los problemas que se originan de este desbalance de clases y trataremos de dar luces a algunas herramientas que nos permitan sortear esos problemas y como medir realmente el desempeño en esta clase de problemas. 

#### 2.a Carga de datos
Descargue los datos y cárguelos usando pandas. Haga una exploración rápida de los datos, cuantos datos hay, como se reparten sus valores.

Grafique la matriz de correlación, a primera vista parece relevante mantener la columna `Time`? En su opinión, conociendo la naturaleza del problema le parece relevante esa información para predecir el _target_?

Cuantos ejemplos hay de cada clase? Son las clases desbalanceadas efectivamente? 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('creditcard.csv')

df.describe()

# . . . 

plt.imshow(df.corr())

# . . .

df.loc[lambda x: x.Class != 0].shape

#### 2.b Ligero preprocesamiento

* Escale la columna `Amount`. Por qué es aconsejable realizar esto? Le parece necesario realizarlo en el resto de las columnas? 

* Separe los atributos del _target_.

* Separe luego los datos en _Training set_ y _Validation set_, con un 20% de los datos como validación y el resto como _train_. Asegurese que se mantienen las proporciones de ejemplos de cada clase en ambos sets. 

* Qué ocurriría si por ejemplo todos los ejemplos de la clase 1 quedaran en el _validation set_, que haría cualquier máquina de aprendizaje al aprender solo con la clase 0?

In [ ]:
from sklearn.preprocessing import StandardScaler

# . . .

df['Amount'] = scaler.transform(df['Amount'].values.reshape(-1,1))

x = df[df.columns[1:-2]]
y = df.Class

from sklearn.model_selection import train_test_split

#### 2.c Regresor logístico

Entrene un regresor logístico con los datos obtenidos de la pregunta anterior y calcule su desempeño (_score_ por ahora) sobre los datos de validación. A priori le parece un buen desempeño? 

In [ ]:
from sklearn.linear_model import LogisticRegression
# . . . 
# . . . 

#### 2.d _Always_ 0

Suponga un modelo le entrega como resultado en el conjunto de validación el siguiente `y_pred`. ¿Qué está haciendo este predictor? ¿Qué _score_ obtendría tal modelo sobre el conjunto de validación? ¿Le parece que el modelo tenga un buen desempeño? Concluya sobre la calidad del _accuracy_ que obtenemos con `.score` para evaluar el desempeño de estos modelos. 

In [ ]:
y_pred = pd.np.zeros(y_val.shape)

# . . .

#### 2.e Matriz de confusión

Investigue un poco sobre los valores que conforman la matriz de confusión o _confusion matrix_. Comente sobre la significancia de los distintos valores para el problema en cuestión. 

Escriba una función que a partir de un modelo o de los valores predecidos por un modelo, grafique la matriz de confusión. 

In [ ]:
# Puede usar libreria seaborn para realizar facilmente heatmaps anotados: 
import seaborn as sns
# . . .
sns.heatmap( confusion_matrix(y_test, model.predict(X_test))/len(X_test),cmap='winter', annot=True)    
# . . .

Grafique la matriz de confusión para la regresión logística realizada en el punto c y para los valores `y_pred` del punto d. Le parecen parecidos ahora ambos modelos? 

#### 2.f Curva ROC
La curva ROC tiene bastante utilidad para evaluar la calidad de distintos modelos en estos casos. Investigue un poco sobre su significado para poder interpretarla correctamente y luego escriba una función que reciba un modelo entrenado y la grafíque. 

Note que `sklearn` trae implementado una función que entrega los puntos de la curva ROC, note eso sí que debe entregarle las probabilidades usando el método `predict_proba` del modelo entrenado. 

Para evaluar distintos modelos utilizará bastante las curvas ROC y la matriz de confusión, por lo que sería recomendable escribir una función que reciba un modelo y datos y realice tanto la curva ROC como la matriz de confusión, aunque escribir tal función no es requisito. 

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_ts, model.predict_proba(x_ts)[:,1],pos_label=1)

plt.plot(fpr, tpr, label=# . . . )

Utilice la función definida para su modelo de regresión logística. Comente el gráfico obtenido. 

#### 2.g Arbol de Clasificación
Entrene un arbol de clasificación para tratar de resolver el problema. Pruebe unas cuantas profundidades máximas distintas y elija una. Si quiere puede modificar los otros hiperparámetros del modelo. 

Evalúe su desempeño respecto a los modelos lineales usando las métricas relevantes y compare su costo computacional. Discuta.

In [ ]:
tree = DecisionTreeClassifier(
            criterion='gini',
            max_depth=3
)
# . . .

#### 2.h _Undersampling_

Considerando que la gran mayoría del aprendizaje se realiza en los ejemplos no fraudulentos, una aproximación para tratar de mejorar el desempeño de los modelos entrenados es simplemente reducir la cantidad de ejemplos de una clase para obtener nuevos conjuntos de datos con clases más balanceadas. Así, el aprendizaje se realiza de manera más balanceada entre ambas clases y se logran extraer mejor los atributos de cada clase. 

Complete la siguiente función para que realice un _undersampling_ de la clase 0 para obtener una proporción en el nuevo conjundo de datos de `times` veces el numero de transacciones normales frente a fraudulentas. Comente sobre la decisión del parámetro `replace` de la función `numpy.random.choice`. 


Note que la función debe recibir los datos de entrenamiento. Se pide que retorne un conjunto de datos de entrenamiento _undersampled_ para poder entrenar. Las validaciones deben realizarse sobre el conjunto de Validación original. 

In [ ]:
def undersample(x_tr, y_tr, times = 10):
    fraud_indices = y_tr.index[y_tr == 1]
    normal_indices = # . . .

    Count_Fraud_transacation = # . . .
    
    undersample_index = pd.np.concatenate([
        pd.np.random.choice(
            normal_indices,
            Count_Fraud_transaction * #something,
            replace = # True or False?)
                            ),
        fraud_indices])
    
    undersample_index = pd.np.concatenate([fraud_indices,Normal_indices_undersample])
    undersampled_x = x_tr.loc[undersample_index]
    # . . .
    return undersampled_x, undersampled_y

#### 2.i Entrenar con _Undersampling_

Pruebe distintos valores para el parámetro `times` y evalue si hay diferencias significativas en la curva ROC y la matriz de confución para el mismo árbol de clasificación entrenado anteriormente. Evidentemente el número de ejemplos de entrenamiento cambia a medida cambia el parámetro `times`, sin embargo nuestro interés no es que tan bien se comporta el modelo sobre los datos de entrenamiento si no sobre los de validación, los cuales deben ser constantes a lo largo del análisis.

¿Como se comparan estos modelos con el modelo con todos los datos (recuerde que las comparaciones deben hacerse calculando el desempeño sobre el conjunto de validación completo con el modelo una vez entrenado)? ¿Ve alguna ventaja en el uso de _undersampling_ frente al uso de todos los datos? 

#### 2.j _Oversampling_

Otra aproximación para resolver el problema de las clases desbalanceadas es hacer _oversampling_. A partir de una muestra relativamente pequeña de ejemplos, en este caso de los casos de fraude, se busca crear un conjunto de datos más grande similar al inicial, que permita al modelo aprender las caracteristicas de esa clase, buscando nuevamente obtener clases más balanceadas que originalmente, pero esta vez sin reducir la cantidad de ejemplos. Esta aproximación tiene la ventaja que no reduce la cantidad de ejemplos pero trae la complicación de decidir cómo se van a crear los datos nuevos.

Dos aproximaciones bastante estandar son SMOTE y ADASYN. Investigue un poco que hace SMOTE y que diferencia implemente ADASYN. Por qué cree puede ser deseable buscar que los datos sintéticos no sean linealmente dependientes de los datos originales? 

Elija alguno de los dos métodos e impelementelo con el árbol entrenado previamente. Comente sobre el desempeño de tal modelo. 

In [ ]:
!pip install  imbalanced-learn

from imblearn.over_sampling import SMOTE, ADASYN
ada = ADASYN()
# . . .
x_tr_r, y_tr_r  = smo.fit_resample(x_tr,y_tr)


Note que tanto SMOTE como ADASYN tienen muchos parámetros que podrían afectar la calidad de los datos generados. No se requiere que elijan tales parametros (los por defecto bastan), pero se valorará un pequeño comentario sobre los parámetros de los métodos y qué representan en la heuristica utilizada. 

#### 2.k Redes neuronales _Feed Forward_ 

Por útimo, probaremos utilizar una red neuronal densa sobre los datos aumentados. Las Redes Neuronales son una herramienta muy poderosa pero comparativamente costosa frente a otros modelos. Entrenar una red neuronal puede resultar muy costos computacionalmente, por su gran numero de parámetros entrenables y por el caracter iterativo de su entrenamiento. Además, el gran numero de parámetros hace que las redes neuronales sean particularmente sensibles al _overfitting_, por lo cual suele ser necesario tener grandes cantidades de datos para poder entrenar una red neuronal obteniendo desempeños que justifiquen su costo adicional.

Por esto, entrenaremos la red neuronal sobre los datos aumentados. Básese en el código siguiente para crea la red neuronal. Explique los parámetros que se pasan en cada etapa de su creación, para esto puede leer la documentación de keras. Finalente, entrene "completamente" la red, es decir hasta que se estanque el error de validación o comience a aumentar. ¿Cómo se compara la red frente a otros modelos, tanto en desempeño como en costo? 

In [ ]:
from keras.layers import Dense
from keras.models import Sequential

model = Sequential()

model.add(Dense(256,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='sgd',
             loss='binary_crossentropy',
             metrics=['accuracy'])
history = model.fit(x_tr.values,y_tr.values, epochs=20, validation_data=(X_test,y_test))

# note that history.history gives you the values of tr and val errors and accuracies